In [ ]:
import sys

if "google.colab" in sys.modules:
    !pip install -q git+https://github.com/theislab/cellrank@dev
    !pip install python-igraph

import os
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
import pandas as pd
import anndata as ad
from matplotlib.colors import ListedColormap

scv.settings.verbosity = 3
cr.settings.verbosity = 2

import warnings

warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

scv.settings.presenter_view = True  # set max width size for presenter view
scv.set_figure_params('scvelo')  # for beautified visualization

In [ ]:
wdir = "/home/ug0302/CITEseq/savedata/cellrank"
os.chdir(wdir)

In [ ]:
i = "PLCPR5"

obs = pd.read_csv(i + "/cellname.txt")
var_names = pd.read_csv(i + "/genename.txt")

X = pd.read_csv(i + "/matrix.txt", sep = "\t", index_col = 0)
splice = pd.read_csv(i + "/splice.txt", sep = "\t", index_col = 0)
unsplice = pd.read_csv(i + "/unsplice.txt", sep = "\t", index_col = 0)

obs.index = obs.x
X.index = obs.x
splice.index = obs.x
unsplice.index = obs.x
var_names.index = X.columns

adata = ad.AnnData(X, obs = obs, var = var_names)
adata.layers["unspliced"] = unsplice
adata.layers["spliced"] = splice

umap = pd.read_csv(i + "/coord_" + i + ".txt", sep = '\t', index_col = 0)
adata.obsm['X_umap'] = umap.values

info = pd.read_csv(i + "/info_protein_" + i + ".txt", sep = '\t', index_col = 0)
adata.obs = pd.concat([adata.obs, info], axis = 1)

scv.pp.filter_and_normalize(adata, min_shared_counts = 20, n_top_genes = 2000)
scv.tl.score_genes_cell_cycle(adata)
sc.pp.regress_out(adata, ['S_score', 'G2M_score'], n_jobs = 10)
scv.pp.moments(adata, n_pcs = 30, n_neighbors = 30)

### Run scVelo

In [ ]:
model = ["deterministic","stochastic","dynamical"]
col = ["#0070b2","#009bc7","#5ec7dd","#b8e3ea","#f3f3f1","#fccdb9","#f79676","#f15e4c","#da1735"]
col = ListedColormap(col)

for j in model:
    scv.tl.recover_dynamics(adata, n_jobs = 50)
    scv.tl.velocity(adata, mode = j)
    scv.tl.velocity_graph(adata)
    
    scv.pl.velocity_embedding_stream(adata, basis = "umap", size = 200, alpha = 1,
                                 color = "CD49f", color_map = col, # matplotlib color map
                                 legend_fontsize = 12, title = "", smooth = 0.9, min_mass = 3,
                                 dpi = 900, figsize = (6,5), save = i + "_" + j + ".png")

### Run CellRank

In [ ]:
cr.tl.initial_states(adata, cluster_key = "clusters", n_states = 1)
cr.pl.initial_states(adata, discrete = True)

In [ ]:
cr.tl.terminal_states(adata, cluster_key="clusters", weight_connectivities=0.2)
cr.pl.terminal_states(adata)

In [ ]:
scv.tl.recover_latent_time(adata, root_key = "initial_states_probs", end_key = "terminal_states_probs")
sc.pl.umap(adata, color = ['latent_time'], size = 100)